In [3]:
import json
import pandas as pd
import spacy
import re
import codecs

import pathlib
from pathlib import Path
import os

import langid


import codecs
import pandas as pd
import re
#nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
import spacy


import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.corpora import Dictionary

import pyLDAvis.gensim_models
import pyLDAvis.sklearn
#pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation



In [4]:
def load_data(path: str):
    with open(path, encoding = 'utf-8') as f:
        dataset = json.load(f)
    return dataset

In [5]:
def extract_title(dataset: pd.DataFrame) -> list:
    titles = []
    for i in dataset['Browser History']:
            try:
                 titles.append(i['title'])
            except KeyError:
                    pass
    return titles

In [6]:
def create_txt(dataset: list, name, path):

    new_path = os.path.join(path, name)
    f = open(new_path, 'a')
    
    for item in dataset:
        without_line_breaks = item.replace("\n", " ")
        without_line_breaks = without_line_breaks.replace("\r", " ")
        lines = without_line_breaks + "\n"
        f.write(lines)
    f.close()

In [7]:
def change_letter(dataset: pd.DataFrame) -> list:
    new_dataset = [re.sub('ø', 'oe', text) for text in dataset]
    new_dataset = [re.sub('æ', 'ae', text) for text in new_dataset]
    new_dataset = [re.sub('å', 'aa', text) for text in new_dataset]
    new_dataset = [re.sub('Ø', 'oe', text) for text in new_dataset]
    new_dataset = [re.sub('Æ', 'ae', text) for text in new_dataset]
    new_dataset = [re.sub('Å', 'aa', text) for text in new_dataset]
    new_dataset = [re.sub('ü', 'ue', text) for text in new_dataset]
    new_dataset = [re.sub('Ü', 'ue', text) for text in new_dataset]
    new_dataset = [re.sub('ä', 'ae', text) for text in new_dataset]
    new_dataset = [re.sub('Ä', 'ae', text) for text in new_dataset]
    new_dataset = [re.sub('ö', 'oe', text) for text in new_dataset]
    new_dataset = [re.sub('Ö', 'oe', text) for text in new_dataset]
    
    return new_dataset

In [8]:
def get_clean_text(dataset: list, stops_da, stops_en) -> list:
    no_urls = [re.sub(r"http\S+", "", text) for text in dataset] 
    only_letters = [re.sub(r"(#[A-Za-z]+)|(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", ' ' , text) for text in no_urls] 
    only_letters = [text.replace('\n', ' ') for text in only_letters]
    lowercased_str = [text.lower().split() for text in only_letters] 
    no_stopwords = [[w for w in text if not w in stops_da] for text in lowercased_str] 
    no_stopwords = [[w for w in text if not w in stops_en] for text in no_stopwords] 
    cleaned_text = [" ".join(text) for text in no_stopwords] 
    return cleaned_text

In [9]:
def tokenize_text(data):
    tokens = [word_tokenize(text) for text in data] 
    return tokens

In [10]:
def lemmatize_text(sent):
    lemmas = [x.lemma_ for x in nlp(sent)]
    return lemmas

In [59]:
def lemmatize_posts(tokenized_data: list):
    lemmas = []
    for post in tokenized_data:
        lemma = [lemmatize_text(x) for x in post]
        lemmas.append([item for sublist in lemma for item in sublist])
    return lemmas

In [60]:
def only_da(dataset: list) -> list:

    langid.set_languages(['da', 'en'])  
    da = []

    for i in dataset:
        lang = langid.classify(i)
        if lang[0] != 'en':          
            da.append(i)


    return da

In [72]:
data = load_data('data/google_data/BrowserHistoryAS.json')

In [73]:
titles = extract_title(data)
print(len(titles))

#print(titles[:20])

13921


In [14]:
#create_txt(titles, name = 'all_titles.txt', path = 'data/google_data/prepared_txt')

In [74]:
da = only_da(titles)
print(len(da))

10388


In [75]:
df = pd.DataFrame(da, columns=['row_posts'])
df.head()

,row_posts
0,Psykolog dømt for seksuelt misbrug af incestof...
1,Indland | Nyheder Indland | Vi holder dig orie...
2,Seneste nyt – Få de seneste nyheder her | DR
3,dr.dk nyheder - Google-søgning
4,dr.dk nyheder - Google-søgning


In [1]:
file = open("stop_words.txt","r+")
stop_words = file.read().split()

In [76]:
nlp = spacy.load("da_core_news_sm")
stopwords_da = stopwords.words("danish")  
stopwords_da.extend(stop_words)

nlp1 = spacy.load("en_core_web_sm")
stopwords_en = stopwords.words("english")

In [77]:
titles_new = change_letter(df['row_posts'])

In [78]:
cleaned_titles = get_clean_text(titles_new, stops_da = stopwords_da, stops_en = stopwords_en)

In [79]:
remove_empty_strings = [string for string in cleaned_titles if string != ""]

In [80]:
tokenized_titles= tokenize_text(remove_empty_strings)

In [81]:

lemmatized_titles = lemmatize_posts(tokenized_titles)

In [86]:
df1 = pd.DataFrame(remove_empty_strings , columns=['cleaned_titles'])

df1['tokenized_titles'] = tokenized_titles
df1['lemmatized_titles'] = lemmatized_titles

In [91]:
#df1.to_pickle('data/google_data/prepared_txt/all_titles_da_df.pkl')

In [92]:
bigram = gensim.models.Phrases(df1['lemmatized_titles'])
posts_bigrams = [bigram[line] for line in df1['lemmatized_titles']]

In [93]:
dictionary = Dictionary(posts_bigrams)
corpus = [dictionary.doc2bow(text) for text in posts_bigrams]

In [94]:
ldamodel = LdaModel(corpus = corpus, 
                    num_topics = 5, 
                    id2word = dictionary,
                    update_every = 1,
                    passes = 10,
                    per_word_topics = True)

In [95]:
pyLDAvis.enable_notebook()
model =pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
#pyLDAvis.display(model)
pyLDAvis.save_html(model, 'data/google_data/models/all_titles_da.html')

/home/kate/.local/share/virtualenvs/myproject-o7xzvy_i/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/home/kate/.local/share/virtualenvs/myproject-o7xzvy_i/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/kate/.local/share/virtualenvs/myproject-o7xzvy_i/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/kate/.local/share/virtualenvs/myproject-o7xzvy_i/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the